In [ ]:
import yasa
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import seaborn as sns
import pandas as pd
sns.set(font_scale=1.2)
import mne
import glob

In [ ]:
PATH = '/Users/ajsimon/Dropbox (Personal)/Data/Overnight PSG/Example/'

#create list of PSG files
FILES = glob.glob(PATH + '*.edf')
FILES.sort()

#create list of hypnogram files  
HYPNO = glob.glob(PATH + '*.csv')
HYPNO.sort()

#For this example, we will use the first file in the path, which is participant 01619
f = 0

In [ ]:
#load PSG data from example participant
eeg = mne.io.read_raw_edf(FILES[f], preload=True)
picks = mne.pick_types(eeg.info, meg=False, eeg=True, eog=True)

#select the channels where eeg was recorded, discard the rest
eeg.pick_channels(['M1-REF','M2-REF','F3-REF','F4-REF','C3-REF','C4-REF','O1-REF','O2-REF'])  # Select a subset of EEG channels

#store EEG data into an e x t matrix, where e = num elecs and t = samples -- for visualization purposes
data = eeg.get_data() 

#convert data from Volts to µV
data = data*1000000

#Display 10 seconds of raw data from F3
fig, ax = plt.subplots(1, 1, figsize=(16, 4))
plt.plot(data[3,200000:204000])
plt.ylabel('Amplitude (uV)')
plt.ylim([-0.000210, -0.000170])

In [ ]:
#preprocess PSG data
# Apply a bandpass filter from 0.3 to 40 Hz 
eeg.filter(0.3, 40)             
    
#Downsample to SF 
SF = 100    #Define frequency to downsample to -- Nyquist rate is 90 Hz for resolving spectral power in frequencies up to 45 Hz (gamma = 30-45 Hz -- per Walsh et al., 2017) 
            #VERY IMPORTANT THAT THE SAMPLING RATE MUST BE KEPT ABOVE THIS
            #Raw data is sampled at 400 Hz -- we don't need to downsample but can if we want to reduce size of data
        
eeg.resample(SF)   
    
#re-reference EEG to linked-mastoids, as opposed to the contralateral mastoid reference that the raw signal is referenced to
eeg.set_eeg_reference(['M1-REF', 'M2-REF'])
    
#select the channels where eeg was recorded, discard the rest
eeg.pick_channels(['F3-REF','F4-REF','C3-REF','C4-REF','O1-REF','O2-REF']) 
chan = eeg.ch_names

#store EEG data into an e x t matrix, where e = num elecs and t = samples -- for visualization purposes
data = eeg.get_data() 

#Display 10 seconds of raw data from F3
fig, ax = plt.subplots(1, 1, figsize=(16, 4))
plt.plot(data[3,50000:51000])
plt.ylabel('Amplitude (uV)')

In [ ]:
### Import hypnogram
hypnog = np.loadtxt(fname = HYPNO[f],dtype = 'str',delimiter = ',',skiprows=5)  
        
#transform hypnogram file into single column vector of stage info
hypnog = hypnog[:,3]
    
#upsample the hypnogram to have the same sampling freq as the EEG data   
hypno_up = yasa.hypno_upsample_to_data(hypno=hypnog, sf_hypno=(1/30), data=data, sf_data=SF)
print(hypno_up.size == data.shape[1])  # Does the hypnogram have the same number of samples as data?
print(hypno_up.size, 'samples:', hypno_up)

#plot spectrogram
fig = yasa.plot_spectrogram(data[0,], SF, hypno=hypno_up, fmax=30, cmap='Spectral_r', trimperc=5)

In [ ]:
### Artifact rejection
art, zscores = yasa.art_detect(data, SF, window=5, method='covar', threshold=3)
art.shape, zscores.shape
    
print(f'{art.sum()} / {art.size} epochs rejected.')

In [ ]:
### Plot the artifact vector
plt.plot(art);
plt.yticks([0, 1], labels=['Good (0)', 'Art (1)']);

In [ ]:
### Plot a histogram of z-score distributions
sns.distplot(zscores)
plt.title('Histogram of z-scores')
plt.xlabel('Z-scores')
plt.ylabel('Density')
plt.axvline(2, color='r', label='Threshold')
plt.axvline(-2, color='r')
plt.legend(frameon=False);

In [ ]:
from scipy.special import erf
threshold = 3
perc_expected_rejected = (1 - erf(threshold / np.sqrt(2))) * 100
print(f'{perc_expected_rejected:.2f}% of all epochs are expected to be rejected.')

# Actual
(art.sum() / art.size) * 100
print(f'{(art.sum() / art.size) * 100:.2f}% of all epochs were actually rejected.')

# The resolution of art is 5 seconds, so its sampling frequency is 1/5 (= 0.2 Hz)
sf_art = 1 / 5
art_up = yasa.hypno_upsample_to_data(art, sf_art, data, SF)
art_up.shape, hypno_up.shape

# Add -1 to hypnogram where artifacts were detected
hypno_with_art = hypno_up.copy()
hypno_with_art[art_up] = -1

# Proportion of each stage in ``hypno_with_art``
pd.Series(hypno_with_art).value_counts(normalize=True)

# Plot new hypnogram and spectrogram on Fz
yasa.plot_spectrogram(data[0, :], SF, hypno_with_art);

In [ ]:
### Slow wave detection
sw = yasa.sw_detect(data, SF, ch_names=chan, hypno=hypno_with_art, include=(2, 3), freq_sw=(0.5, 1.6))

# Get the average per channel and stage
sw.summary(grp_chan=True, grp_stage=True, aggfunc='mean')

# Plot an average template of the detected slow-waves, centered around the negative peak
ax = sw.plot_average(center="NegPeak", time_before=0.4, time_after=0.8, palette="Set1")
ax.legend(frameon=False)
sns.despine()